In [4]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import urllib.request
import zipfile
import os
from notebookutils import mssparkutils

StatementMeta(, 55c2fcdf-526a-4703-90ae-55660a766118, 6, Finished, Available, Finished)

In [5]:
# 1. Gerar lista de meses mes atual -2 até -11 (os ultimos 12 meses tirando os 2 ultimos meses)
ontem = datetime.today() - timedelta(days=1)
ultimos_meses = [(ontem - relativedelta(months=i+2)).strftime("%Y%m") for i in range(10)]

StatementMeta(, 55c2fcdf-526a-4703-90ae-55660a766118, 7, Finished, Available, Finished)

In [6]:
print(ultimos_meses)

StatementMeta(, 55c2fcdf-526a-4703-90ae-55660a766118, 8, Finished, Available, Finished)

['202505', '202504', '202503', '202502', '202501', '202412', '202411', '202410', '202409', '202408']


In [7]:
# 2. Criar pasta temporária local
tmp_base = "/tmp/inf_diario"
os.makedirs(tmp_base, exist_ok=True)

StatementMeta(, 55c2fcdf-526a-4703-90ae-55660a766118, 9, Finished, Available, Finished)

In [8]:
# 3. Loop para processar cada mês
for anomes in ultimos_meses:
    print(f"🔄 Processando {anomes}...")

    try:
        # 3.1. Montar URL e paths
        url = f"https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{anomes}.zip"
        local_zip_path = f"{tmp_base}/inf_diario_fi_{anomes}.zip"
        extract_path = f"{tmp_base}/{anomes}"
        os.makedirs(extract_path, exist_ok=True)

        # 3.2. Baixar o ZIP
        urllib.request.urlretrieve(url, local_zip_path)

        # 3.3. Extrair o CSV
        with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

        # 3.4. Pegar nome do CSV extraído
        csv_file = [f for f in os.listdir(extract_path) if f.endswith(".csv")][0]
        local_csv_path = os.path.join(extract_path, csv_file)

        # 3.5. Ler conteúdo e salvar no Lakehouse
        with open(local_csv_path, "r", encoding="latin1") as f:
            content = f.read()

        dest_path = f"Files/bronze/fi_m-2-m-11_meses/inf_diario_fi_{anomes}.csv"
        mssparkutils.fs.put(dest_path, content, overwrite=True)
        print(f"✅ Sucesso ao processar {anomes}")

    except Exception as e:
        print(f"❌ Erro ao processar {anomes}: {e}")

StatementMeta(, 55c2fcdf-526a-4703-90ae-55660a766118, 10, Finished, Available, Finished)

🔄 Processando 202505...
✅ Sucesso ao processar 202505
🔄 Processando 202504...
✅ Sucesso ao processar 202504
🔄 Processando 202503...
✅ Sucesso ao processar 202503
🔄 Processando 202502...
✅ Sucesso ao processar 202502
🔄 Processando 202501...
✅ Sucesso ao processar 202501
🔄 Processando 202412...
✅ Sucesso ao processar 202412
🔄 Processando 202411...
✅ Sucesso ao processar 202411
🔄 Processando 202410...
✅ Sucesso ao processar 202410
🔄 Processando 202409...
✅ Sucesso ao processar 202409
🔄 Processando 202408...
✅ Sucesso ao processar 202408
